# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Final Project Group 196: Prelude to Milestone 4

**Harvard University**<br/>
**Fall 2024**<br/>
**Group Members & Awesome People**: Zachary Sardi-Santos, Sarah Mrad, Adam Latif, Samaga Pokharel

<hr style="height:2.4pt">

# 🎓 PROBLEM STATEMENT #



The SP500, or Standard & Poor's 500, is a stock market index tracking the stock performance of 500 of the largest companies listed on stock exchanges in the United States. Investors in the SP500 index face a problem trying to estimate their return on investment based on the plethora of microeconomic and macroeconomic factors that influence the index. Thus, given their investing circumstances and knowledge, they invest inappropriate amounts of capital and do not maximize their earning potential.

We will solve this problem by building prediction models using many economic, financial, and stock-market-related features to predict monthly returns on the SP500 index. Our final model will be used as a tool for these investors to accurately predict future returns on the index given current and future projected financial scenarios. 

# 🎓 Creating and Processing the DATA 🕳️

We begin this milestone by importing a series of csv files as dataframes representing our response (SP500 monthly returns) and feature variables. Then, we perform many data processing actions with the goal of merging the dataframes into one giant design matrix (+ our response variable). 

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import resample
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold



In [6]:
consumer_price_index = pd.read_csv("Monthly_Data/Consumer Price Index 1947-2024.csv")

In [7]:
consumer_sentiment = pd.read_csv("Monthly_Data/Consumer_Sentiment_Umich_1952-2024.csv")
federal_funds_rate = pd.read_csv("Monthly_Data/FederalFunds 1954-2024.csv")
labor_force_rate = pd.read_csv("Monthly_Data/Labor_Force_Participation_Rate_1948-2024.csv")
gdp = pd.read_csv("Monthly_Data/BBKI_real_gdp_growth.csv")
m2 = pd.read_csv("Monthly_Data/M2_1959-2024.csv")
market_yield_10year = pd.read_csv("Monthly_Data/Market Yield 10-Year 1962-2024.csv")
mortgage_rate = pd.read_csv("Monthly_Data/Mortgage_Rate_1971-2024.csv")
msci = pd.read_csv("Monthly_Data/MSCI_World_Index_1978-2024.csv")
savings_rate = pd.read_csv("Monthly_Data/Personal_Savings_Rate_1959-2024.csv")
sp500_monthly_returns = pd.read_csv("Monthly_Data/SP500_monthly_returns_1985-2024.csv")
sp500_pe_ratio = pd.read_csv("Monthly_Data/SP500_PE_Ratio.csv")
total_vehicle_sales = pd.read_csv("Monthly_Data/Total_Vehicle_Sales_1976-2024.csv")
unemployment_rate = pd.read_csv("Monthly_Data/Unemployment Rate 1948-2024.csv")


In [8]:
dataset_dict = {'consumer_sentiment':consumer_sentiment, 'federal_funds_rate': federal_funds_rate, 
'gdp':gdp , 'labor_force_rate':labor_force_rate, 'm2':m2, 'market_yield_10year':market_yield_10year, 
'mortgage_rate':mortgage_rate, 'msci':msci, 'savings_rate':savings_rate, 
'sp500_monthly_returns':sp500_monthly_returns, 'sp500_pe_ratio': sp500_pe_ratio, 
 'total_vehicle_sales': total_vehicle_sales, 
'unemployment_rate': unemployment_rate}

In [9]:
dataset_list = [consumer_sentiment, federal_funds_rate, gdp, labor_force_rate, m2, market_yield_10year
               , mortgage_rate, msci, savings_rate, sp500_monthly_returns,
               sp500_pe_ratio, total_vehicle_sales, unemployment_rate]

In [10]:
def lookup(dictionary):
    for (key, value) in dictionary.items():
        # x = pd.DataFrame(x)
        print(f"This is {key}")
        print(value.head())

lookup(dataset_dict)

This is consumer_sentiment
         DATE UMCSENT
0  1952-11-01    86.2
1  1952-12-01       .
2  1953-01-01       .
3  1953-02-01    90.7
4  1953-03-01       .
This is federal_funds_rate
         DATE  FEDFUNDS
0  1954-07-01      0.80
1  1954-08-01      1.22
2  1954-09-01      1.07
3  1954-10-01      0.85
4  1954-11-01      0.83
This is gdp
         DATE    BBKMGDP
0  1960-01-01  15.736726
1  1960-02-01   5.944251
2  1960-03-01  -4.395573
3  1960-04-01  -3.817574
4  1960-05-01  -3.394048
This is labor_force_rate
         DATE  CIVPART
0  1948-01-01     58.6
1  1948-02-01     58.9
2  1948-03-01     58.5
3  1948-04-01     59.0
4  1948-05-01     58.3
This is m2
         DATE   M2SL
0  1959-01-01  286.6
1  1959-02-01  287.7
2  1959-03-01  289.2
3  1959-04-01  290.1
4  1959-05-01  292.2
This is market_yield_10year
         DATE     DGS10
0  1962-01-01  4.083182
1  1962-02-01  4.039444
2  1962-03-01  3.930455
3  1962-04-01  3.843000
4  1962-05-01  3.873636
This is mortgage_rate
         DATE 

Now we must normalize the date colums of the dataframes, so we will be able to concatenate later on

first, actually have to standardize the date name to date, b/c some date colums
are named differently

In [11]:
#function takes in a list of dataframes
def standardize_column_names(dataframes):
    #create empty list to store new dataframes in after function is called on them
    standardized_dfs = []
    for df in dataframes:
        #convert the column names to lowercase
        df.columns = df.columns.str.lower()  
        #add converted df to list
        standardized_dfs.append(df)
    return standardized_dfs

#perform function on the list of dataframes I created
standardize_column_names(dataset_list)


[           date umcsent
 0    1952-11-01    86.2
 1    1952-12-01       .
 2    1953-01-01       .
 3    1953-02-01    90.7
 4    1953-03-01       .
 ..          ...     ...
 850  2023-09-01    67.8
 851  2023-10-01    63.8
 852  2023-11-01    61.3
 853  2023-12-01    69.7
 854  2024-01-01      79
 
 [855 rows x 2 columns],
            date  fedfunds
 0    1954-07-01      0.80
 1    1954-08-01      1.22
 2    1954-09-01      1.07
 3    1954-10-01      0.85
 4    1954-11-01      0.83
 ..          ...       ...
 839  2024-06-01      5.33
 840  2024-07-01      5.33
 841  2024-08-01      5.33
 842  2024-09-01      5.13
 843  2024-10-01      4.83
 
 [844 rows x 2 columns],
            date    bbkmgdp
 0    1960-01-01  15.736726
 1    1960-02-01   5.944251
 2    1960-03-01  -4.395573
 3    1960-04-01  -3.817574
 4    1960-05-01  -3.394048
 ..          ...        ...
 772  2024-05-01   3.736513
 773  2024-06-01   3.069704
 774  2024-07-01   2.929177
 775  2024-08-01   2.497320
 776  2024-09-

Now I need to normalize the date columns to have them all in the same format, eg 1948-01-01

In [12]:
#takes in a list of dataframes and the actual date column of the specific dataframe
def normalize_date_columns(dataframes, date_column_name):
#same process as standardizing the date column title
    normalized_dfs = []
    for df in dataframes:
        if date_column_name in df.columns:
            df[date_column_name] = pd.to_datetime(df[date_column_name], errors='coerce')
            normalized_dfs.append(df)
        else:
            print(f"Warning: Column '{date_column_name}' not found in one of the DataFrames.")
    return normalized_dfs


normalize_date_columns(dataset_list, 'date' )

/var/folders/yq/5p00hdm16xsb4kbmhwk9rcx80000gn/T/ipykernel_10206/1753152597.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[date_column_name] = pd.to_datetime(df[date_column_name], errors='coerce')


[          date umcsent
 0   1952-11-01    86.2
 1   1952-12-01       .
 2   1953-01-01       .
 3   1953-02-01    90.7
 4   1953-03-01       .
 ..         ...     ...
 850 2023-09-01    67.8
 851 2023-10-01    63.8
 852 2023-11-01    61.3
 853 2023-12-01    69.7
 854 2024-01-01      79
 
 [855 rows x 2 columns],
           date  fedfunds
 0   1954-07-01      0.80
 1   1954-08-01      1.22
 2   1954-09-01      1.07
 3   1954-10-01      0.85
 4   1954-11-01      0.83
 ..         ...       ...
 839 2024-06-01      5.33
 840 2024-07-01      5.33
 841 2024-08-01      5.33
 842 2024-09-01      5.13
 843 2024-10-01      4.83
 
 [844 rows x 2 columns],
           date    bbkmgdp
 0   1960-01-01  15.736726
 1   1960-02-01   5.944251
 2   1960-03-01  -4.395573
 3   1960-04-01  -3.817574
 4   1960-05-01  -3.394048
 ..         ...        ...
 772 2024-05-01   3.736513
 773 2024-06-01   3.069704
 774 2024-07-01   2.929177
 775 2024-08-01   2.497320
 776 2024-09-01   1.412970
 
 [777 rows x 2 colum

Now I must rearrange all the date columns in ascending order. Actually, not sure if I must, but just for safety reasons, I will do so. Maybe it will make the merging easier. 

In [13]:
updated_dataset_list = []
def ascending_dates(list):
    for dataset in list:
        dataset = dataset.sort_values(by='date', ascending = True)
        # df_sorted = df.sort_values(by='col1', ascending=True)
        print(dataset.head())
        updated_dataset_list.append(dataset)

ascending_dates(dataset_list)


        date umcsent
0 1952-11-01    86.2
1 1952-12-01       .
2 1953-01-01       .
3 1953-02-01    90.7
4 1953-03-01       .
        date  fedfunds
0 1954-07-01      0.80
1 1954-08-01      1.22
2 1954-09-01      1.07
3 1954-10-01      0.85
4 1954-11-01      0.83
        date    bbkmgdp
0 1960-01-01  15.736726
1 1960-02-01   5.944251
2 1960-03-01  -4.395573
3 1960-04-01  -3.817574
4 1960-05-01  -3.394048
        date  civpart
0 1948-01-01     58.6
1 1948-02-01     58.9
2 1948-03-01     58.5
3 1948-04-01     59.0
4 1948-05-01     58.3
        date   m2sl
0 1959-01-01  286.6
1 1959-02-01  287.7
2 1959-03-01  289.2
3 1959-04-01  290.1
4 1959-05-01  292.2
        date     dgs10
0 1962-01-01  4.083182
1 1962-02-01  4.039444
2 1962-03-01  3.930455
3 1962-04-01  3.843000
4 1962-05-01  3.873636
        date  mortgage30us
0 1971-04-01        7.3100
1 1971-05-01        7.4250
2 1971-06-01        7.5300
3 1971-07-01        7.6040
4 1971-08-01        7.6975
        date    msci world
0 1978-12-01 

So now I will make sure that all the datasets end at the same date, which is 2024-01-01. 

In [14]:
  
def last_date(df, target_date):
    target_date = pd.to_datetime(target_date)  # make sure date is datetime object in case
    # drop rows where the date is greater than the target date, so last one will be 2024-01-01
    df = df[df['date'] <= target_date]
    return df

target_date = '2024-01-01'

# last_date(unemployment_rate, '2024-01-01')
# last_date(gdp, '2024-01-01')

for i, dataset in enumerate(updated_dataset_list):
    updated_dataset_list[i] = last_date(dataset, target_date)
    print(updated_dataset_list[i].tail())  # verify the last row is 2024-01-01 using .tail() attribute

          date umcsent
850 2023-09-01    67.8
851 2023-10-01    63.8
852 2023-11-01    61.3
853 2023-12-01    69.7
854 2024-01-01      79
          date  fedfunds
830 2023-09-01      5.33
831 2023-10-01      5.33
832 2023-11-01      5.33
833 2023-12-01      5.33
834 2024-01-01      5.33
          date   bbkmgdp
764 2023-09-01  4.363554
765 2023-10-01  2.390753
766 2023-11-01  2.977604
767 2023-12-01  1.625261
768 2024-01-01 -0.375014
          date  civpart
908 2023-09-01     62.8
909 2023-10-01     62.7
910 2023-11-01     62.8
911 2023-12-01     62.5
912 2024-01-01     62.5
          date     m2sl
776 2023-09-01  20681.4
777 2023-10-01  20662.5
778 2023-11-01  20675.7
779 2023-12-01  20725.4
780 2024-01-01  20725.9
          date     dgs10
740 2023-09-01  4.379000
741 2023-10-01  4.798095
742 2023-11-01  4.502857
743 2023-12-01  4.020000
744 2024-01-01  4.058095
          date  mortgage30us
629 2023-09-01        7.2000
630 2023-10-01        7.6200
631 2023-11-01        7.4420
632 2023

Now I must deal with the sp500_monthly_returns dataframe, as it has more than two columns. I believe I can split these columns up into multiple dataframes (to use as features) and Additionally, I will calculate the monthly return in percentages via the closing price column. Specifically, the return for the next month will be the next month's closing price divided by the previous month's closing price. I use the closing price because this is typically how returns of a stock or index are calculated, using the closing prices.

In [15]:
sp500_monthly_returns

,date,open,high,low,close,adj close,volume
0,2024-11-01,5723.22,6017.31,5696.51,5969.34,5969.34,68825900000
1,2024-10-01,5757.73,5878.46,5674,5705.45,5705.45,82412430000
2,2024-09-01,5623.89,5767.37,5402.62,5762.48,5762.48,79564830000
3,2024-08-01,5537.84,5651.62,5119.26,5648.4,5648.4,81097300000
4,2024-07-01,5471.08,5669.67,5390.95,5522.3,5522.3,80160390000
...,...,...,...,...,...,...,...
474,1985-05-01,179.83,189.98,178.35,189.55,189.55,"2,350,340,000"
475,1985-04-01,180.66,183.61,177.86,179.83,179.83,"1,981,880,000"
476,1985-03-01,181.18,183.89,176.53,180.66,180.66,"2,153,090,000"
477,1985-02-01,179.63,183.95,177.75,181.18,181.18,"2,194,620,000"


In [16]:
sp500_monthly_returns = sp500_monthly_returns.sort_values(by='date')

In [17]:
sp500_monthly_returns.columns

Index(['date', 'open', 'high', 'low', 'close ', 'adj close ', 'volume'], dtype='object')

In [18]:
print(sp500_monthly_returns['close '].dtype)

object


So I need to convert the column entries of the close column to floats. First, I need to remove the commas. Then I must convert the entries to floats. This is easy to do as I can just apply pandas functions to an entire column using pd attributes.

In [19]:
#need to convert entries to integers
# df['column_name'] = df['column_name'].astype(int)
#first I need 
# Remove commas from the 'col1' column
# df['col1'] = df['col1'].str.replace(',', '')
sp500_monthly_returns['close '] = sp500_monthly_returns['close '].str.replace(',', '')
sp500_monthly_returns['close '] = sp500_monthly_returns['close '].astype(float)

In [20]:
sp500_monthly_returns['percent_change'] = sp500_monthly_returns['close '].pct_change() * 100
sp500_monthly_returns['percent_change']

478         NaN
477    0.862885
476   -0.287007
475   -0.459427
474    5.405105
         ...   
4      1.132135
3      2.283469
2      2.019687
1     -0.989678
0      4.625227
Name: percent_change, Length: 479, dtype: float64

In [21]:
sp500_percentage = pd.DataFrame(sp500_monthly_returns['percent_change'])

In [22]:
sp500_percentage.insert(0,'date', sp500_monthly_returns['date'])
#inserting date column into the percentage dataframe

In [23]:
sp500_percentage

,date,percent_change
478,1985-01-01,NaN
477,1985-02-01,0.862885
476,1985-03-01,-0.287007
475,1985-04-01,-0.459427
474,1985-05-01,5.405105
...,...,...
4,2024-07-01,1.132135
3,2024-08-01,2.283469
2,2024-09-01,2.019687
1,2024-10-01,-0.989678


In [24]:
#setting NaN value = 0 as this is the first year of the return calculation
sp500_percentage['percent_change']= sp500_percentage['percent_change'].fillna(0)

In [25]:
sp500_percentage.head()

,date,percent_change
478,1985-01-01,0.000000
477,1985-02-01,0.862885
476,1985-03-01,-0.287007
475,1985-04-01,-0.459427
474,1985-05-01,5.405105


In [26]:
sp500_monthly_volume = pd.DataFrame(sp500_monthly_returns['volume'])

In [27]:
sp500_monthly_volume.insert(0,'date', sp500_monthly_returns['date'])

In [28]:
sp500_monthly_volume.head()

,date,volume
478,1985-01-01,"2,673,710,000"
477,1985-02-01,"2,194,620,000"
476,1985-03-01,"2,153,090,000"
475,1985-04-01,"1,981,880,000"
474,1985-05-01,"2,350,340,000"


In [29]:
sp500_monthly_high = pd.DataFrame(sp500_monthly_returns['high'])

In [30]:
sp500_monthly_high.insert(0,'date', sp500_monthly_returns['date'])

In [31]:
sp500_monthly_high.head()

,date,high
478,1985-01-01,180.27
477,1985-02-01,183.95
476,1985-03-01,183.89
475,1985-04-01,183.61
474,1985-05-01,189.98


In [32]:
sp500_monthly_low = pd.DataFrame(sp500_monthly_returns['low'])
sp500_monthly_low.insert(0,'date', sp500_monthly_returns['date'])
sp500_monthly_low.head()

,date,low
478,1985-01-01,163.36
477,1985-02-01,177.75
476,1985-03-01,176.53
475,1985-04-01,177.86
474,1985-05-01,178.35


Now I must have all the datasets start at the same date. Seems like 1997-01-01 may be the date to choose, as this is the minimum date in the price adjusted gdp dataframe (gepucurrent)

In [33]:
updated_dataset_list.append(sp500_percentage)
def first_date(df, target_date):  
    # df should only include rows on and after the target_date so 1985-01-01 is first date
    df = df[df['date'] >= target_date]
    return df

target_date = '1997-01-01'

# apply function to the list CHATGPT CODE 
updated_dataframes = [first_date(df, target_date) for df in updated_dataset_list]

# see if changes actually occured
for df in updated_dataframes:
    print(df.head())  # Print the first few rows of each updated DataFrame

          date umcsent
530 1997-01-01    97.4
531 1997-02-01    99.7
532 1997-03-01   100.0
533 1997-04-01   101.4
534 1997-05-01   103.2
          date  fedfunds
510 1997-01-01      5.25
511 1997-02-01      5.19
512 1997-03-01      5.39
513 1997-04-01      5.51
514 1997-05-01      5.50
          date   bbkmgdp
444 1997-01-01  0.284662
445 1997-02-01  4.986687
446 1997-03-01  6.322837
447 1997-04-01  7.677815
448 1997-05-01  6.596759
          date  civpart
588 1997-01-01     67.0
589 1997-02-01     66.9
590 1997-03-01     67.1
591 1997-04-01     67.1
592 1997-05-01     67.1
          date    m2sl
456 1997-01-01  3834.6
457 1997-02-01  3846.3
458 1997-03-01  3861.2
459 1997-04-01  3877.0
460 1997-05-01  3889.2
          date     dgs10
420 1997-01-01  6.579048
421 1997-02-01  6.419474
422 1997-03-01  6.694500
423 1997-04-01  6.885455
424 1997-05-01  6.710952
          date  mortgage30us
309 1997-01-01        7.8240
310 1997-02-01        7.6500
311 1997-03-01        7.8975
312 1997-04-01

Before merging, just need to get some little errands done and clean the respective dataframes 

In [34]:
sp500_monthly_volume['volume'] = sp500_monthly_volume['volume'].str.replace(',', '')
sp500_monthly_low['low'] = sp500_monthly_low['low'].str.replace(',', '')
sp500_monthly_high['high']= sp500_monthly_high['high'].str.replace(',', '')

In [35]:
from functools import reduce
merged_df = reduce(lambda left, right: pd.merge(left, right, on='date', how='outer'), updated_dataframes)

# Sort by date
merged_df.sort_values(by='date', inplace=True)

# Reset the index
merged_df.reset_index(drop=True, inplace=True)

merged_df

,date,umcsent,fedfunds,bbkmgdp,civpart,m2sl,dgs10,mortgage30us,msci world,psavert,open,high,low,close,adj close,volume,value,totalsa,unrate,percent_change
0,1997-01-01,97.4,5.25,0.284662,67.0,3834.6,6.579048,7.8240,102815.407672,5.9,740.74,794.67,729.55,786.16,786.16,"11,635,830,000",19.53,15.682,5.3,6.131706
1,1997-02-01,99.7,5.19,4.986687,66.9,3846.3,6.419474,7.6500,103978.699659,6.0,786.16,817.68,773.43,790.82,790.82,"9,715,930,000",20.09,15.271,5.2,0.592755
2,1997-03-01,100.0,5.39,6.322837,67.1,3861.2,6.694500,7.8975,101902.554128,6.1,790.82,814.9,756.13,757.12,757.12,"10,120,760,000",19.69,15.816,5.2,-4.261400
3,1997-04-01,101.4,5.51,7.677815,67.1,3877.0,6.885455,8.1425,105214.055674,6.2,757.12,804.13,733.54,801.34,801.34,"10,454,880,000",18.94,15.058,5.1,5.840554
4,1997-05-01,103.2,5.50,6.596759,67.1,3889.2,6.710952,7.9440,111689.040528,6.5,801.34,851.87,793.21,848.28,848.28,"10,106,650,000",20.60,15.088,4.9,5.857688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,2024-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.132135
331,2024-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.283469
332,2024-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.019687
333,2024-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.989678


In [36]:
merged_df.iloc[-10:,0]

325   2024-02-01
326   2024-03-01
327   2024-04-01
328   2024-05-01
329   2024-06-01
330   2024-07-01
331   2024-08-01
332   2024-09-01
333   2024-10-01
334   2024-11-01
Name: date, dtype: datetime64[ns]

In [37]:
merged_df = merged_df.iloc[:-10,:]

In [38]:
merged_df

,date,umcsent,fedfunds,bbkmgdp,civpart,m2sl,dgs10,mortgage30us,msci world,psavert,open,high,low,close,adj close,volume,value,totalsa,unrate,percent_change
0,1997-01-01,97.4,5.25,0.284662,67.0,3834.6,6.579048,7.8240,102815.407672,5.9,740.74,794.67,729.55,786.16,786.16,"11,635,830,000",19.53,15.682,5.3,6.131706
1,1997-02-01,99.7,5.19,4.986687,66.9,3846.3,6.419474,7.6500,103978.699659,6.0,786.16,817.68,773.43,790.82,790.82,"9,715,930,000",20.09,15.271,5.2,0.592755
2,1997-03-01,100.0,5.39,6.322837,67.1,3861.2,6.694500,7.8975,101902.554128,6.1,790.82,814.9,756.13,757.12,757.12,"10,120,760,000",19.69,15.816,5.2,-4.261400
3,1997-04-01,101.4,5.51,7.677815,67.1,3877.0,6.885455,8.1425,105214.055674,6.2,757.12,804.13,733.54,801.34,801.34,"10,454,880,000",18.94,15.058,5.1,5.840554
4,1997-05-01,103.2,5.50,6.596759,67.1,3889.2,6.710952,7.9440,111689.040528,6.5,801.34,851.87,793.21,848.28,848.28,"10,106,650,000",20.60,15.088,4.9,5.857688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,2023-09-01,67.8,5.33,4.363554,62.8,20681.4,4.379000,7.2000,565837.313861,4.4,4530.60,4541.25,4238.63,4288.05,4288.05,73482980000,23.93,16.193,3.8,-4.871929
321,2023-10-01,63.8,5.33,2.390753,62.7,20662.5,4.798095,7.6200,549419.087402,4.5,4284.52,4393.57,4103.78,4193.80,4193.80,83519460000,22.78,15.799,3.8,-2.197969
322,2023-11-01,61.3,5.33,2.977604,62.8,20675.7,4.502857,7.4420,600928.286726,4.6,4201.27,4587.6,4197.74,4567.80,4567.80,80970570000,23.51,15.945,3.7,8.917926
323,2023-12-01,69.7,5.33,1.625261,62.5,20725.4,4.020000,6.8150,630430.407194,4.4,4559.43,4793.30,4546.5,4769.83,4769.83,81530670000,24.35,16.386,3.7,4.422917


In [39]:
merged_df.rename(columns={'umcsent': 'consumer_sentiment', 'fedfunds':'federal_funds_rate'}, inplace=True)

/var/folders/yq/5p00hdm16xsb4kbmhwk9rcx80000gn/T/ipykernel_10206/3898417585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.rename(columns={'umcsent': 'consumer_sentiment', 'fedfunds':'federal_funds_rate'}, inplace=True)


In [40]:
merged_df = merged_df.rename(columns={'civpart':'labor_force_participation_rate', 'gepucurrent':'gdp', 'm2sl':'m2', 'dgs10':'yield_10_year',
                 'mortgage30us':'mortgage_30_year', 'open':'sp500_monthly_open', 'low':'sp500_monthly_low', 'high':'sp500_monthly_high', 'close':'sp500_monthly_close', 'msci world':'msci', 'psavert':'savings_rate',
                 'value':'sp500_pe_ratio', 'volume':'sp500_monthly_volume', 'totalsa':'vehicle_sales','unrate':'unemployment_rate'})

In [41]:
merged_df

,date,consumer_sentiment,federal_funds_rate,bbkmgdp,labor_force_participation_rate,m2,yield_10_year,mortgage_30_year,msci,savings_rate,sp500_monthly_open,sp500_monthly_high,sp500_monthly_low,close,adj close,sp500_monthly_volume,sp500_pe_ratio,vehicle_sales,unemployment_rate,percent_change
0,1997-01-01,97.4,5.25,0.284662,67.0,3834.6,6.579048,7.8240,102815.407672,5.9,740.74,794.67,729.55,786.16,786.16,"11,635,830,000",19.53,15.682,5.3,6.131706
1,1997-02-01,99.7,5.19,4.986687,66.9,3846.3,6.419474,7.6500,103978.699659,6.0,786.16,817.68,773.43,790.82,790.82,"9,715,930,000",20.09,15.271,5.2,0.592755
2,1997-03-01,100.0,5.39,6.322837,67.1,3861.2,6.694500,7.8975,101902.554128,6.1,790.82,814.9,756.13,757.12,757.12,"10,120,760,000",19.69,15.816,5.2,-4.261400
3,1997-04-01,101.4,5.51,7.677815,67.1,3877.0,6.885455,8.1425,105214.055674,6.2,757.12,804.13,733.54,801.34,801.34,"10,454,880,000",18.94,15.058,5.1,5.840554
4,1997-05-01,103.2,5.50,6.596759,67.1,3889.2,6.710952,7.9440,111689.040528,6.5,801.34,851.87,793.21,848.28,848.28,"10,106,650,000",20.60,15.088,4.9,5.857688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,2023-09-01,67.8,5.33,4.363554,62.8,20681.4,4.379000,7.2000,565837.313861,4.4,4530.60,4541.25,4238.63,4288.05,4288.05,73482980000,23.93,16.193,3.8,-4.871929
321,2023-10-01,63.8,5.33,2.390753,62.7,20662.5,4.798095,7.6200,549419.087402,4.5,4284.52,4393.57,4103.78,4193.80,4193.80,83519460000,22.78,15.799,3.8,-2.197969
322,2023-11-01,61.3,5.33,2.977604,62.8,20675.7,4.502857,7.4420,600928.286726,4.6,4201.27,4587.6,4197.74,4567.80,4567.80,80970570000,23.51,15.945,3.7,8.917926
323,2023-12-01,69.7,5.33,1.625261,62.5,20725.4,4.020000,6.8150,630430.407194,4.4,4559.43,4793.30,4546.5,4769.83,4769.83,81530670000,24.35,16.386,3.7,4.422917


In [42]:
# merged_df.columns
# merged_df = merged_df.drop('adj close ', axis = 1)
merged_df.head()

,date,consumer_sentiment,federal_funds_rate,bbkmgdp,labor_force_participation_rate,m2,yield_10_year,mortgage_30_year,msci,savings_rate,sp500_monthly_open,sp500_monthly_high,sp500_monthly_low,close,adj close,sp500_monthly_volume,sp500_pe_ratio,vehicle_sales,unemployment_rate,percent_change
0,1997-01-01,97.4,5.25,0.284662,67.0,3834.6,6.579048,7.8240,102815.407672,5.9,740.74,794.67,729.55,786.16,786.16,"11,635,830,000",19.53,15.682,5.3,6.131706
1,1997-02-01,99.7,5.19,4.986687,66.9,3846.3,6.419474,7.6500,103978.699659,6.0,786.16,817.68,773.43,790.82,790.82,"9,715,930,000",20.09,15.271,5.2,0.592755
2,1997-03-01,100.0,5.39,6.322837,67.1,3861.2,6.694500,7.8975,101902.554128,6.1,790.82,814.9,756.13,757.12,757.12,"10,120,760,000",19.69,15.816,5.2,-4.261400
3,1997-04-01,101.4,5.51,7.677815,67.1,3877.0,6.885455,8.1425,105214.055674,6.2,757.12,804.13,733.54,801.34,801.34,"10,454,880,000",18.94,15.058,5.1,5.840554
4,1997-05-01,103.2,5.50,6.596759,67.1,3889.2,6.710952,7.9440,111689.040528,6.5,801.34,851.87,793.21,848.28,848.28,"10,106,650,000",20.60,15.088,4.9,5.857688


In [43]:
merged_df['sp500_monthly_volume']= merged_df['sp500_monthly_volume'].str.replace(',','')
merged_df = merged_df.drop('adj close ', axis=1)
merged_df = merged_df.drop('date',axis=1)

In [44]:
#ChatGPT code dealing with this error
for col in merged_df.columns:
    if merged_df[col].dtype == 'object':  # Only apply to string columns
        merged_df[col] = merged_df[col].str.replace(',', '', regex=True)  # Remove commas
        merged_df[col] = merged_df[col].str.replace('[^0-9.-]', '', regex=True)  # Remove non-numeric characters

# Convert the entire DataFrame to float
merged_df = merged_df.astype(float)

In [45]:
merged_df.head()

,consumer_sentiment,federal_funds_rate,bbkmgdp,labor_force_participation_rate,m2,yield_10_year,mortgage_30_year,msci,savings_rate,sp500_monthly_open,sp500_monthly_high,sp500_monthly_low,close,sp500_monthly_volume,sp500_pe_ratio,vehicle_sales,unemployment_rate,percent_change
0,97.4,5.25,0.284662,67.0,3834.6,6.579048,7.8240,102815.407672,5.9,740.74,794.67,729.55,786.16,1.163583e+10,19.53,15.682,5.3,6.131706
1,99.7,5.19,4.986687,66.9,3846.3,6.419474,7.6500,103978.699659,6.0,786.16,817.68,773.43,790.82,9.715930e+09,20.09,15.271,5.2,0.592755
2,100.0,5.39,6.322837,67.1,3861.2,6.694500,7.8975,101902.554128,6.1,790.82,814.90,756.13,757.12,1.012076e+10,19.69,15.816,5.2,-4.261400
3,101.4,5.51,7.677815,67.1,3877.0,6.885455,8.1425,105214.055674,6.2,757.12,804.13,733.54,801.34,1.045488e+10,18.94,15.058,5.1,5.840554
4,103.2,5.50,6.596759,67.1,3889.2,6.710952,7.9440,111689.040528,6.5,801.34,851.87,793.21,848.28,1.010665e+10,20.60,15.088,4.9,5.857688


In [46]:
merged_df.to_csv('merged_df.csv')